## RETRIEVAL OF RAW HIGHWAY CRASH DATA

### Default Import

In [1]:
from ris import pysqldb
from IPython.display import Markdown, clear_output
from sqlalchemy import create_engine
import ris
import getpass
import datetime 
import pandas as pd
import numpy as np
import os
import requests

timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
ts = datetime.datetime.now().strftime('%Y-%m-%d')

Markdown('<strong>Notebook run on: {} | by {} | Using ris library version: {} | File Location: {}'.format(
    timestamp, getpass.getuser(), ris.__version__, os.getcwd()
))


<strong>Notebook run on: 2021-06-16 13:35 | by soge | Using ris library version: 2.2.1 | File Location: E:\RIS\Staff Folders\Samuel\Requests\SH\HBKBQE

### Db connections

In [2]:
try: 
    db.params['user']
except:
    db = pysqldb.DbConnect(type='PG', server='dotdevrhpgsql01', database='ris')


Additional database connection details required:
User name (ris):soge
Password (ris)········


In [3]:
data =pd.read_excel('BQE Four Crossings Crash Data Request.xlsx')

### Reference Markers 

In [4]:
ref_mkr=pd.read_excel('BQE Four Crossings Crash Data Request.xlsx', sheet_name= 'Reference Marker')

In [5]:
markers = tuple(map(str,list(ref_mkr[ref_mkr.columns[6]][2:])))

In [6]:
print markers

('278IX2M23089', '278IX2M24089', '278IX2M23088', '278IX2M24088', '278IX2M23087', '278IX2M24087', '278IX2M23086', '278IX2M24086', '278IX2M23085', '278IX2M24085', '278IX2M23084', '278IX2M24084', '278IX2M23083', '278IX2M24083', '278IX2M23082', '278IX2M24082', '278IX2M23081', '278IX2M24081', '278IX2M23080', '278IX2M24080', '278IX2M23079', '278IX2M24079', '278IX2M23078', '278IX2M24078')


### NYSDOT Data

In [7]:
raw_crashes = db.dfquery("""SELECT ogc_fid, gid, case_num, case_yr, ref_mrkr, accd_dte, road_sys, 
                               num_of_fat, num_of_inj, reportable, police_dep, intersect_, muni, 
                               precinct, num_of_veh, accd_typ, locn, traf_cntl, light_cond, 
                               weather, road_char, road_surf_, collision_, ped_loc, ped_actn, 
                               ext_of_inj, regn_cnty_, accd_tme, rpt_agcy, dmv_accd_c, traf_way, 
                               rdway_acc_, err_cde, comm_veh_a, highway_in, intersect1, utm_northi, 
                               utm_eastin, crashid  
                            FROM nysdot_all
                            WHERE ref_mrkr in {}
                            and case_yr::int between 2016 and 2018

                            """.format(markers))

In [8]:
#crashids
rc = tuple(list(raw_crashes.crashid))

In [9]:
raw_vehicle = db.dfquery("""SELECT *  
                            FROM nysdot_vehicle
                            WHERE crashid in {c}
                            and case_yr between 2016 and 2018
                            """.format(c=rc))



In [10]:
raw_contributing = db.dfquery("""SELECT *  
                                 FROM nysdot_appfactor
                                 WHERE crashid in {c}
                                 and case_yr between 2016 and 2018
                                 """.format(c=rc))



In [11]:
raw_crashes.to_csv('hbkbqe_raw_crashes_{}.csv'.format(ts),index=False)

In [12]:
raw_vehicle.to_csv('hbkbqe_raw_vehicle_{}.csv'.format(ts),index=False)

In [13]:
raw_contributing.to_csv('hbkbqe_raw_contributing_{}.csv'.format(ts),index=False)